### Начало

In [8]:
#папка с библиотеками (для колаба)
base = 'drive/My Drive/GPB'

In [9]:
import sys
import re

import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
!pip install -Uqq ipdb
import ipdb # %pdb on для включения дебаг режима
from matplotlib import style
style.use('ggplot')
%config InlineBackend.figure_format = 'retina'

in_colab = 'google.colab' in sys.modules

if in_colab:
    sys.path.append(base)

def get_drive_path(path):
    in_colab = 'google.colab' in sys.modules
    return '/content/'+base+'/'+re.sub(r'\\','/',path) if in_colab else path

In [10]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install torch-geometric
!pip install torch-geometric-temporal
!pip install dgl
!pip install memory_profiler

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


In [11]:
import os
import pickle

import torch
from torch_geometric.data import GraphSAINTRandomWalkSampler, ClusterData, ClusterLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import gnn_package as gnn

### Proteins dataset

In [12]:
dataset_dir = get_drive_path('data\\graph\\PROTEINS')

df_edge_index = pd.read_csv( os.path.join(dataset_dir, 'PROTEINS_A.txt'), header=None)
df_graphs_indicator = pd.read_csv( os.path.join(dataset_dir, 'PROTEINS_graph_indicator.txt'), header=None)[0]
df_graphs_labels = pd.read_csv( os.path.join(dataset_dir, 'PROTEINS_graph_labels.txt'), header=None)[0]
df_attrs = pd.read_csv( os.path.join(dataset_dir, 'PROTEINS_node_attributes.txt'), header=None)[0]
df_node_labels = pd.read_csv( os.path.join(dataset_dir, 'PROTEINS_node_labels.txt'), header=None)[0]

In [39]:
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

graphs_id = df_graphs_indicator.unique()
for graph_id in tqdm(graphs_id):
    graph_nodes = df_graphs_indicator[df_graphs_indicator == graph_id].index
    encoder = LabelEncoder()
    
    edge_index = df_edge_index[df_edge_index.isin(graph_nodes)].to_numpy()
    edge_index[:,0] = encoder.fit_transform(edge_index[:,0])
    edge_index[:,1] = encoder.fit_transform(edge_index[:,1])
    edge_index = torch.LongTensor(edge_index.T)

    node_features = df_attrs[df_attrs.index.isin(graph_nodes)]
    node_features = torch.Tensor(node_features.to_numpy())

    y = df_node_labels[df_node_labels.index.isin(graph_nodes)]
    y = torch.Tensor(y.to_numpy())

    graph_nodes = encoder.fit_transform(graph_nodes)

    data = gnn.data.torch_geometric_graph(x=node_features, edge_index=edge_index, edge_attr=None,
                                 y=y)

KeyboardInterrupt: ignored

> /usr/local/lib/python3.7/dist-packages/pandas/core/frame.py(9226)isin()
   9224                 )
   9225             return self._constructor(
-> 9226                 algorithms.isin(self.values.ravel(), values).reshape(self.shape),
   9227                 self.index,
   9228                 self.columns,

ipdb> exit


In [17]:
graph_nodes_grouped = []
last_graph_id = 1
last_graph_nodes_group = []
for node,graph_id in enumerate(df_graphs_indicator, 1):
    if graph_id == last_graph_id:
        last_graph_nodes_group.append(node)
    else:
        graph_nodes_grouped.append(np.array(last_graph_nodes_group))
        last_graph_nodes_group = [node]
        last_graph_id += 1
graph_nodes_grouped.append(np.array(last_graph_nodes_group))

In [18]:
edge_index_grouped = []
last_edge_index_group = []
last_graph_id = 0
for i, (node1, node2) in df_edge_index.iterrows():
    if node1 in graph_nodes_grouped[last_graph_id]:
        last_edge_index_group.append((node1, node2))
    else:
        edge_index_grouped.append(np.array(last_edge_index_group))
        last_edge_index_group = [(node1, node2)]
        last_graph_id += 1